In [8]:
from utils import*

In [231]:

####################################################################################################################################################
class NODE(object):
    ################################################################################################################################################
    def __init__(self, markers_color, IDD=None, port=None):
        self.markers_color = markers_color
        self.IDD = IDD
        self.port = port
    ################################################################################################################################################
    def get_data(self, dataset_name, file_name, ref_node=None, window_length=11, ref_node_data=None):        
        data = self.get_motion(dataset_name, file_name, window_length=window_length, ref_node_data=ref_node_data)  

        if self.IDD is not None: 
            rssi = self.get_rssi(dataset_name, file_name, window_length=window_length)
            data = data.merge( rssi, on='time', how='outer', suffixes=('', ''), sort=True )

        if self.port is not None: 
            vind = self.get_vind(dataset_name, file_name, window_length=window_length)
            data = data.merge( vind, on='time', how='outer', suffixes=('', ''), sort=True )

        return data
    ################################################################################################################################################
    def get_motion(self, dataset_name, file_name, window_length=11, ref_node_data=None):   
        markers_file_path = get_markers_file_path(dataset_name, file_name)  
        raw_df  = pd.read_csv(
            markers_file_path,                                                  # relative python path to subdirectory
            usecols = ['time', self.markers_color],                             # Only load the three columns specified.
            parse_dates = ['time'] ) 
        
        # DON'T USE!! markers are switched and smoothing causes error    
        # Processing
        # self.markers = self.markers.rolling(window_length, axis=0).mean()    # Gap filling
        # self.markers = self.markers.ffill(axis=0).bfill(axis=0)              # Smoothing

        # Time
        date_time = pd.to_datetime( raw_df['time'] , format=datime_format)
        time = [ np.round( (datetime.combine(date.min, t.time())-datetime.min).total_seconds(), 2) for t in date_time]
        
        # Markers
        markers = [list(map(float, l.replace(']','').replace('[','').replace('\n','').split(", "))) for l in raw_df[self.markers_color].values]  
        markers_npy = np.array(markers).reshape(len(time), -1, 3)

        # Center    
        center = np.mean(markers_npy, axis=1)         
        center = np.nan_to_num(center)
        center = signal.savgol_filter( center, window_length=window_length, polyorder=1, axis=0)            
        
        # Norm
        norm = np.cross( markers_npy[:,1,:] - markers_npy[:,0,:], markers_npy[:,2,:] - markers_npy[:,0,:])
        # norm[ norm[:,2]<0, :] *= -1 
        norm = np.nan_to_num(norm)
        # norm = signal.savgol_filter( norm, window_length=window_length, polyorder=1, axis=0)  
        norm = norm / ( np.reshape(np.linalg.norm(norm, axis=1), (-1,1)) * np.ones((1,3)))

        # Relative movements 
        if ref_node_data is None:            
            return pd.DataFrame({
                'time': time,
                'markers': markers,
                'center': list(center), 
                'norm': list(norm)
                })    
                   
        N = 10
        ref_center = np.tile( np.mean(ref_node_data.center.loc[:N], axis=0), (len(ref_node_data),1) )
        ref_norm = np.tile( np.mean(ref_node_data.norm.loc[:N], axis=0), (len(ref_node_data),1) )            

        distance_vec = ref_center - center

        # Distance
        distance = np.linalg.norm( distance_vec, axis=1)
        distance = signal.savgol_filter( distance, window_length=window_length, polyorder=polyorder, axis=0)                        

        # Lateral Misalignment
        lat_misalignment = np.sqrt(distance**2 - np.sum(np.multiply( distance_vec, ref_norm), axis=1)**2)           
        lat_misalignment = signal.savgol_filter( lat_misalignment, window_length=window_length, polyorder=1, axis=0)        

        # Angular Misalignment
        ang_misalignment = np.arcsin( np.linalg.norm(np.cross(ref_norm, norm), axis=1) )*180/np.pi
        ang_misalignment = signal.savgol_filter( ang_misalignment, window_length=window_length, polyorder=1, axis=0)  

        return pd.DataFrame({
            'time': time,
            'markers': markers,
            'center': list(center), 
            'norm': list(norm),
            'distance': list(distance),
            'lat_misalignment': list(lat_misalignment),
            'ang_misalignment': list(ang_misalignment)
            })                 
    ################################################################################################################################################
    def get_rssi(self, dataset_name, file_name, window_length=11):
        # Load data 
        rfid_file_path = get_rfid_file_path(dataset_name, file_name)       
        raw_df = pd.read_csv(
            rfid_file_path,                                                     # relative python path to subdirectory
            delimiter  = ';',                                                   # Tab-separated value file.
            usecols = ['IDD', 'Time', 'Ant/RSSI'],                              # Only load the three columns specified.
            parse_dates = ['Time'] )                                            # Intepret the birth_date column as a date      

        # self.rssi.loc[ self.rssi.IDD != self.IDD,'rssi'] = np.nan
        raw_df = raw_df.loc[ raw_df['IDD'] == self.IDD, :]

        # Time
        date_time = pd.to_datetime( raw_df['Time'] , format=datime_format)
        time = [ np.round( (datetime.combine(date.min, t.time())-datetime.min).total_seconds(), 2) for t in date_time]
        
        # RSSI
        # rssi_df = pd.DataFrame({ 'rssi': raw_df['Ant/RSSI'].str.replace('Ant.No 1 - RSSI: ', '').astype(int) })
        rssi_df = raw_df['Ant/RSSI'].str.replace('Ant.No 1 - RSSI: ', '').astype(int) 
        rssi_df = rssi_df.rolling(window_length, axis=0).median()   # Smoothing
        rssi_df = rssi_df.ffill(axis=0).bfill(axis=0)               # Gap Filling

        return pd.DataFrame({
            'time':time,
            'rssi':rssi_df.tolist() 
            })
    ################################################################################################################################################
    def get_vind(self, dataset_name, file_name, window_length=11):
        # Load data 
        arduino_file_path = get_arduino_file_path(dataset_name, file_name)               
        raw_df = pd.read_csv(arduino_file_path)

        # Time
        date_time = pd.to_datetime( raw_df['time'] , format=datime_format)
        time = [ np.round( (datetime.combine(date.min, t.time())-datetime.min).total_seconds(), 2) for t in date_time]

        # V_induced (Vind)
        vind = raw_df[self.port]
        vind = vind.rolling(window_length, axis=0).median()   # Smoothing
        vind = vind.ffill(axis=0).bfill(axis=0)               # Gap Filling
         
        return pd.DataFrame({
            'time':time,
            'vind':vind.tolist()
        })
########################################################################################################################################################################################################################################################################################################
class SYSTEM(object):
    ################################################################################################################################################
    def __init__(self, system_info = None):
        self.reader = None
        self.tags = list()

        if system_info is not None:
            system_info_ = system_info.copy()
            reader_markers_color = system_info_.pop('reader')
            self.add_reader(reader_markers_color)
            for IDD,markers_color in system_info_.items(): self.add_tag(markers_color, IDD)
        return
    ################################################################################################################################################
    def add_reader(self, reader_markers_color):
        self.reader = NODE( reader_markers_color )
        return
    ################################################################################################################################################
    def add_tag(self, markers_color, IDD=None, port=None):
        self.tags.append( NODE(markers_color, IDD=IDD, port=port) )   
        return               
    ################################################################################################################################################
    def get_data(self, dataset_name, file_name):
        reader_data = self.reader.get_data(dataset_name, file_name)  
        data = pd.DataFrame({'time':reader_data.time})

        for i, tag in enumerate(self.tags):
            tag_data = tag.get_data(dataset_name, file_name, ref_node_data=reader_data)
            data = data.merge( tag_data, on='time', how='outer', suffixes=('_'+str(i), '' ), sort=True )
        return data
    ################################################################################################################################################
    def get_rssi_data_(self, window_length=11, polyorder=1):
        rssi = pd.DataFrame({'time':self.tags[0].rssi.time})
        for i, tag in enumerate(self.tags):
            tag_rssi = sys.tags[i].rssi.rename({'rssi':'rssi_'+str(i)}, axis=1)
            rssi = rssi.merge( tag_rssi, on='time', how='outer', suffixes=('', ''), sort=True )               
 
        # RSSI data status
        status = pd.DataFrame({'time':rssi.time})
        for column in rssi.columns:
            if column != 'time': status[column.replace('rssi','status')] = np.isnan(rssi[column].values)
        
        # # Smoothing
        rssi.loc[:, rssi.columns != 'time'] = rssi.loc[:, rssi.columns != 'time'].rolling(window_length, axis=0).mean().fillna(method='ffill', axis=0).bfill(axis=0)      
        for col in rssi.columns:
            if col != 'time': rssi.loc[:,col]= signal.savgol_filter( rssi.loc[:,col], window_length=window_length, polyorder=polyorder) 

        return rssi, status       
    ################################################################################################################################################
    def get_data_(self, window_length=11, save=False,  data_status=False):
        rssi, status = self.get_rssi_data()
        motion = self.get_motion_data()       
    
        data = pd.DataFrame({'time':rssi.time})
        data = data.merge( motion, on='time', how='outer', suffixes=('', ''), sort=True )
        data = data.interpolate(method='nearest')        
        data.loc[:, data.columns!='time'] = data.loc[:, data.columns!='time'].rolling(window_length, axis=0).mean().fillna(method='ffill', axis=0).bfill(axis=0)      
        
        data = data.merge(rssi, on='time', how='inner', suffixes=('', ''), sort=True)
        data = data.merge(status, on='time', how='inner', suffixes=('', ''), sort=True)

        time = rssi.time
        time = time[ motion.time.iloc[0] < time]
        time = time[ time < motion.time.iloc[-1] ]
        data = data.merge( time, on='time', how='inner', suffixes=('', ''), sort=True )

        if save:
            dataset_file_path = get_dataset_file_path(self.dataset_name, self.file_name)
            create_folder(dataset_file_path)
            data.to_pickle( dataset_file_path )  
            print(self.file_name, 'is saved.')
        return data
####################################################################################################################################################


In [115]:
dataset_name = 'dataset_04'
file_name = 'record_' + "{0:0=2d}".format(0)

In [202]:
# reader = NODE('red', port ='/dev/cu.usbserial-1420')
reader = NODE('red')
tag = NODE('blue', IDD='E0022400028402EB')


In [232]:
reader_data = reader.get_data(dataset_name, file_name)
tag_data = tag.get_data(dataset_name, file_name, ref_node_data=reader_data)


In [233]:
dataset_name = 'dataset_04'    
rfid_info = get_rfid_info(dataset_name)    
sys = SYSTEM(system_info=rfid_info)

In [234]:
sys.get_data(dataset_name, file_name)

,time,markers_1,center_1,norm_1,distance_1,lat_misalignment_1,ang_misalignment_1,rssi_1,markers,center,norm,distance,lat_misalignment,ang_misalignment,rssi
0,42539.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0
1,42539.89,NaN,NaN,NaN,NaN,NaN,NaN,145.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,42539.97,NaN,NaN,NaN,NaN,NaN,NaN,145.0,NaN,NaN,NaN,NaN,NaN,NaN,145.0
3,42540.04,NaN,NaN,NaN,NaN,NaN,NaN,145.0,NaN,NaN,NaN,NaN,NaN,NaN,145.0
4,42540.12,NaN,NaN,NaN,NaN,NaN,NaN,145.0,NaN,NaN,NaN,NaN,NaN,NaN,145.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,42580.44,NaN,NaN,NaN,NaN,NaN,NaN,137.0,NaN,NaN,NaN,NaN,NaN,NaN,132.0
979,42580.45,"[0.022708884, -0.04082682, 1.1400707, 0.063857...","[0.02041735442424242, 0.011697043789696979, 1....","[-0.41457474464142463, 0.15962850642876183, 0....",0.36747,0.353006,7.000799,NaN,"[-0.17014605, -0.061242323, 1.0733571000000002...","[-0.1778992462121212, -0.01580290620454545, 1....","[-0.3379786169854513, -0.019459284850517518, 0...",0.408001,0.398528,2.689606,NaN
980,42580.52,NaN,NaN,NaN,NaN,NaN,NaN,137.0,NaN,NaN,NaN,NaN,NaN,NaN,132.0
981,42580.59,NaN,NaN,NaN,NaN,NaN,NaN,138.0,NaN,NaN,NaN,NaN,NaN,NaN,132.0


In [213]:
sys.reader.get_data(dataset_name, file_name)

,time,markers,center,norm
0,42540.45,"[-0.09492123126983644, 0.2333592176437378, 1.5...","[-0.09150217913768503, 0.07360810960050332, 1....","[0.1150611391610628, 0.14320878729219003, -0.9..."
1,42540.60,"[-0.08876042068004608, -0.10791151970624924, 1...","[-0.09218531921505924, 0.07451368853111157, 1....","[-0.1809600699682404, -0.19069533982558615, 0...."
2,42540.67,"[-0.08879346400499344, -0.10795783996582033, 1...","[-0.09286845929243344, 0.07541926746171983, 1....","[-0.044270402280562365, 0.18320802466799008, -..."
3,42540.74,"[-0.08878348022699356, -0.10789870470762253, 1...","[-0.09355159936980764, 0.07632484639232807, 1....","[-0.1532079786204611, -0.18541198332531178, 0...."
4,42540.81,"[-0.08875345438718796, -0.1079026386141777, 1....","[-0.09423473944718185, 0.07723042532293632, 1....","[-0.16797844221087543, -0.18496389398569707, 0..."
...,...,...,...,...
498,42579.73,"[-0.08772580325603485, -0.10949791967868804, 1...","[-0.09461528187278993, 0.07406207625754177, 1....","[-0.8868556785433026, 0.05425497634026313, -0...."
499,42580.04,"[-0.09337521344423294, 0.23703770339488986, 1....","[-0.09576910250735551, 0.07404493720016696, 1....","[0.15946505535855285, 0.16329238816551897, -0...."
500,42580.26,"[-0.09339800477027893, 0.2362472414970398, 1.5...","[-0.09692292314192108, 0.07402779814279214, 1....","[0.1547850836258059, 0.17663871652121893, -0.9..."
501,42580.36,"[-0.09333539754152298, 0.2360520362854004, 1.5...","[-0.09807674377648665, 0.07401065908541733, 1....","[0.13917989446954657, 0.17916758563456137, -0...."


0                                                    NaN
1                                                    NaN
2                                                    NaN
3                                                    NaN
4                                                    NaN
                             ...                        
977                                                  NaN
978    [0.02041735442424242, 0.011697043789696979, 1....
979                                                  NaN
980                                                  NaN
981                                                  NaN
Name: center, Length: 982, dtype: object

In [47]:
# dataset_name = 'dataset_00'

# time_folder_path = main_directory + '/dataset/' + dataset_name + '/kinect/time'
# markers_folder_path = main_directory + '/dataset/' + dataset_name + '/kinect/markers'

# for n in range(30):
#     file_name = 'record_' + "{0:0=2d}".format(n)
    
#     time_file_path = time_folder_path + '/' + file_name + '.txt'   
#     with open(time_file_path , 'r') as f:  lines = f.read().splitlines() 
#     date_time = pd.to_datetime( lines , format=datime_format)
#     time = pd.DataFrame({'time':[t.time() for t in date_time]})
             
#     locations_dict_ = dict()
#     for color in ['red','blue','green']:
#         markers_file_path = markers_folder_path + '/' + file_name + '_' + color +'.txt' 
#         locations = pd.read_csv( markers_file_path, delimiter="\t", header=None, dtype=np.float64).to_numpy()
# #         locations_dict_.update({color: locations})
#         locations_dict_.update({color: list(map(list,locations))})
    
#     locations_dict = defaultdict(list)  
#     for l in range(len(time)):
#         for color in ['red','blue','green']:
#             locations_dict[color].append( locations_dict_[color][l] )                            

#   # MArkers (time and locations)
#     markers = pd.concat([time, pd.DataFrame(locations_dict)], axis=1)

#     # Save 
#     markers_file_path = get_markers_file_path(dataset_name, file_name)
#     create_folder(markers_file_path)
#     markers.to_csv(markers_file_path, index=False)
        